In [6]:
from datetime import datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

import skforecast
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster
import shap


## Load Dataset

In [7]:
ts= pd.read_csv('../../../data/ts_top100_prediction.csv')
print(ts.shape)
ts.sample(1)

(1739323, 6)


,date,id,item,sales,income,event
42340,2011-04-05,SUPERMARKET_3_574_BOS_3,SUPERMARKET_3_574,7,50.232,0


### Transfor _date_ columns to _datetime_ format and create **weekday** column (exogenous variable)

In [8]:
ts['date']= pd.to_datetime(ts['date'], format='%Y-%m-%d')
ts['weekday'] = ts['date'].dt.day_name()
ts.head(1)

,date,id,item,sales,income,event,weekday
0,2011-01-29,ACCESORIES_1_108_NYC_1,ACCESORIES_1_108,4,64.4784,0,Saturday


### Create **store_code** columns to filter by store

In [9]:
ts['store_code']=ts['id'].apply(lambda x:x[-5:])
ts.head(2)

,date,id,item,sales,income,event,weekday,store_code
0,2011-01-29,ACCESORIES_1_108_NYC_1,ACCESORIES_1_108,4,64.4784,0,Saturday,NYC_1
1,2011-01-29,HOME_&_GARDEN_1_027_NYC_1,HOME_&_GARDEN_1_027,4,26.2000,0,Saturday,NYC_1


### Run model in top 100 products

In [11]:
# List to save results
resultados = []
store = 'BOS_1'

# Copy of original DataFrame
df_original = ts.copy()

# Select products by item
productos = list(df_original['item'].unique())

for producto in productos:
    # Filter by store_code 
    df_filtrado = df_original[(df_original['store_code'] == store) & (df_original['item'] == producto)]
    
    # Verify if there are enough products
    if len(df_filtrado) < 30 + 14:  # Need at least 44 products
        print(f"Warning: There is not enough data for the product {producto}. Skip")
        continue
    
    # Prepare data
    ts_predict = df_filtrado[['date', 'income', 'weekday', 'event']]
    ts_predict = pd.get_dummies(data=ts_predict, columns=['weekday'], dtype=int)
    ts_predict['date'] = pd.to_datetime(ts_predict['date'])
    ts_predict.sort_values('date', ascending=True, inplace=True)
    ts_predict.set_index('date', inplace=True)
    ts_predict = ts_predict.asfreq('D')

    # Separate test and train dataset, and  also include exogenous variables
    y, exog = ts_predict['income'], ts_predict.drop(columns=['income'])
    y_train, y_test = y[:-30], y[-30:]
    exog_train, exog_test = exog[:-30], exog[-30:]

    # Initializing the model
    forecaster = ForecasterAutoreg(
        regressor=RandomForestRegressor(random_state=15,
                                        max_depth=10,
                                        min_samples_leaf=2,
                                        min_samples_split=5,
                                        n_estimators=50),
        lags=7
    )
    
    # Fit the model
    forecaster.fit(y=y_train, exog=exog_train)

    # Prepare for prediction(next mounth)
    start_date = '2016-04-25' 
    end_date ='2016-05-30'
    # Create range date
    date_range = pd.date_range(start=start_date, end=end_date)
    # Create DataFrame
    df1 = pd.DataFrame(date_range, columns=['date'])
    df1['date'] = pd.to_datetime(df1['date'])
    df1['weekday'] = df1['date'].dt.day_name()
    df1['event'] = 0
    df1 = pd.get_dummies(data=df1, columns=['weekday'], dtype=int)
    df1.sort_values('date', inplace=True, ascending=True)
    df1.set_index('date', inplace=True)
    df1 = df1.asfreq('D')
    exog_test = pd.concat([exog_test, df1], ignore_index=False)
    
    # Predictions
    predicciones = forecaster.predict(steps=60, exog=exog_test)
    
    # Store results
    resultados.append({
        'producto': producto,
        'prediccion': predicciones.values,
        'test': y_test.values
    })
    
    #print(f"Predicction generates to the product: {producto}")


c:\Users\harol\Nuclio\DS-MARKET-TFM\env\Lib\site-packages\sklearn\preprocessing\_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
c:\Users\harol\Nuclio\DS-MARKET-TFM\env\Lib\site-packages\sklearn\preprocessing\_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
c:\Users\harol\Nuclio\DS-MARKET-TFM\env\Lib\site-packages\sklearn\preprocessing\_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
c:\Users\harol\Nuclio\DS-MARKET-TFM\env\Lib\site-packages\sklearn\preprocessing\_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
c:\Users\har

In [12]:
tiendas_inventario = pd.DataFrame(resultados)
tiendas_inventario['test_suma'] = tiendas_inventario['test'].apply(sum)
suma = tiendas_inventario['test_suma'].sum()
tiendas_inventario['prediccion'] = tiendas_inventario['prediccion'].apply(lambda x: np.round(x, decimals=0))
tiendas_inventario['tiendas_inventario_test'] = tiendas_inventario['prediccion'].apply(lambda x: sum(x[:30]))
tiendas_inventario['tiendas_inventario_prediction'] = tiendas_inventario['prediccion'].apply(lambda x: sum(x[30:]))
tiendas_inventario[['producto','test_suma','tiendas_inventario_test','tiendas_inventario_prediction']]

,producto,test_suma,tiendas_inventario_test,tiendas_inventario_prediction
0,ACCESORIES_1_108,397.0582,109.0,99.0
1,HOME_&_GARDEN_1_027,223.8750,319.0,318.0
2,HOME_&_GARDEN_1_053,1178.8875,1300.0,1149.0
3,HOME_&_GARDEN_1_140,1366.0125,805.0,746.0
4,HOME_&_GARDEN_1_177,200.3875,426.0,382.0
...,...,...,...,...
95,SUPERMARKET_3_499,783.3600,965.0,921.0
96,ACCESORIES_1_158,4829.0172,4257.0,4212.0
97,SUPERMARKET_3_282,1345.5360,1449.0,1455.0
98,ACCESORIES_1_354,580.7046,2963.0,3010.0


In [13]:
total_inventario_productos_real = tiendas_inventario['test_suma'].sum()
total_inventario_productos_predicho = tiendas_inventario['tiendas_inventario_test'].sum()
print(total_inventario_productos_real)
print(total_inventario_productos_predicho)
print(total_inventario_productos_real - total_inventario_productos_predicho)

92833.74810000001
90930.0
1903.7481000000116


In [14]:
tienda =  pd.DataFrame(resultados)
file_name = f"{store}_income.xlsx"
tienda.to_excel(file_name)